In [34]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import os

np.random.seed(42)
tf.random.set_seed(42)

In [30]:
# Definir o caminho para o diretório contendo as imagens de treino e teste
data_dir = 'dataset/Kather_texture_2016_image_tiles_5000'
samples, labels = list(), list()

for nome_pasta in os.listdir(data_dir):
    for nome_arquivo in os.listdir(os.path.join(data_dir, nome_pasta)):
        samples.append(os.path.join(nome_pasta,nome_arquivo))
        labels.append(int(nome_pasta[:2]))



In [43]:
all_data = pd.DataFrame({'path':samples,'label':labels})
all_data['label'] = all_data['label'].astype('str')

In [46]:
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.9,  # 20% dos dados serão usados para validação
                             horizontal_flip=True)

In [47]:
# Configuração dos parâmetros da CNN
input_shape = (28, 28, 3)  # Dimensões da imagem de entrada
num_classes = 8  # Número de classes no seu conjunto de dados
batch_size = 32

# Criando os geradores de dados de treinamento e validação usando flow_from_dataframe
train_generator = datagen.flow_from_dataframe(dataframe=all_data,
                                              directory=data_dir,
                                              x_col="path",
                                              y_col="label",
                                              subset="training",  # Usando os dados de treinamento
                                              batch_size=32,
                                              target_size=(28, 28),  # Tamanho da imagem para o modelo MobileNetV2
                                              class_mode="categorical")  # Se for um problema de classificação multiclasse

valid_generator = datagen.flow_from_dataframe(dataframe=all_data,
                                              directory=data_dir,
                                              x_col="path",
                                              y_col="label",
                                              subset="validation",  # Usando os dados de validação
                                              batch_size=32,
                                              target_size=(28, 28),
                                              class_mode="categorical")


Found 500 validated image filenames belonging to 8 classes.


Found 4500 validated image filenames belonging to 8 classes.


In [22]:
np.unique(np.array(train_labels))

array([1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:



# Preprocessamento e aumento de dados para o conjunto de treino
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Escala os valores dos pixels para o intervalo [0, 1]
    shear_range=0.2,        # Aplica shear (cisalhamento) aleatório
    zoom_range=0.2,         # Aplica zoom aleatório
    horizontal_flip=True    # Realiza flip horizontal aleatório
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'  # Modo de classificação categórica para problemas de classificação
)

# Construir o modelo CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compilar o modelo
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Treinar o modelo
model.fit(train_generator, epochs=10)

# Avaliar o modelo no conjunto de teste (não fornecido neste exemplo)
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#     test_data_dir,
#     target_size=input_shape[:2],
#     batch_size=batch_size,
#     class_mode='categorical'
# )
# accuracy = model.evaluate(test_generator)[1]
# print(f'Teste de precisão: {accuracy}')

# Salvar o modelo treinado
model.save('cnn_model.h5')
